# Imports

In [39]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
from uuid import uuid4

# Functions

In [40]:
def runFullParse (inputPiece, iterations=5):
    (corpus, piece) = inputPiece
    jsonPath = "outputs/"+corpus+"/"+piece+".json"
    cmd = ["stack","run","fullParse","--","-n",str(iterations), corpus, piece, "All"] 
    print("Running command: " + (" ".join(cmd)))
    print("Expecting results in " + jsonPath)
    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(cmd,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:")
        print(res.stderr)
        return;
    else:
        f = open(jsonPath)
        results = json.load(f)
        f.close()
        return results
     
def get_corpus_pieces(corpus):
    return sorted (os.path.basename(d).split(".")[0] for d in glob("inputs/slices/"+corpus+"/*.csv"))

def get_corpi():
    return [os.path.basename(d) for d in glob("inputs/slices/*")]

def run_experiment():
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            runFullParse((corpus, piece))

In [41]:
def algo_experiment():
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for algorithm in ["RandomWalk", "RandomSample", "RandomReduction"]:
                commands.append([ corpus, piece, algorithm])

    run_Experiment(commands)


In [42]:
def run_Experiment(commands):
    id = datetime.now().strftime('%Y%m-%d%H-%M%S-') + str(uuid4())

    commands = [ ["stack","run","fullParse","--","-n",str(1), "-id", str(id)] + command for command in commands]
    with Pool(10) as p:
        p.map(run_command, commands)


In [43]:
def run_command(command):
    print("\nRunning command: " + (" ".join(command)))    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(command,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:")
        print(res.stderr)
        return;


# Run Experiments

In [ ]:
if __name__ == '__main__':
    algo_experiment()


Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n01op18-1_01 RandomWalk
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n02op18-2_02 RandomSample
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n03op18-3_03 RandomReduction
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n05op18-5_01 RandomWalk
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n06op18-6_02 RandomSample
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n09op59-3_01 RandomWalk
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f4-39ee-4d28-999e-27efc6f7d122 ABC n10op74_02 RandomSample
Running command: stack run fullParse -- -n 1 -id 202304-1615-1815-a93d54f